# Meta-Feature evaluation

# Imports

In [ ]:
from subpred.transporter_dataset import create_dataset
from subpred.compositions import calculate_paac, calculate_aac
from subpred.eval import (
    get_independent_test_set,
    optimize_hyperparams,
    preprocess_pandas,
    models_quick_compare,
    get_confusion_matrix,
    get_classification_report,
    full_test,
    quick_test
)
from subpred.util import get_feature_score, perform_pca
from subpred.plots import labeled_clustermap, pca_plot_2d, pca_plot_3d
from subpred.pssm import calculate_pssms_notebook

import pandas as pd

# Dataset

In [ ]:
outliers = (
    ["Q9HBR0", "Q07837"]  + ["O81775", "Q9SW07", "Q9FHH5", "Q8S8A0", "Q3E965", "Q3EAV6", "Q3E8L0"]
    
)
df = create_dataset(
    keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
    keywords_component_filter=["Transmembrane"],
    keywords_transport_filter=["Transport"],
    input_file="../data/raw/swissprot/uniprot-reviewed_yes.tab.gz",
    multi_substrate="integrate",
    verbose=True,
    tax_ids_filter=[3702, 9606, 559292],
    output_log="../logs/meta_amino_sugar_dataset.log",
    outliers=outliers,
    sequence_clustering=70
)
taxid_to_organism = {
    3702: "A. thaliana",
    9606: "Human",
    559292: "Yeast",
}
df = df.assign(organism=df.organism_id.map(taxid_to_organism))


# Feature generation

In [ ]:
labels = df.keywords_transport
labels.value_counts()

In [ ]:
df_aac = calculate_aac(df.sequence)
df_paac = calculate_paac(df.sequence)
df_pssm = calculate_pssms_notebook(df.sequence)

In [ ]:
df_combined = pd.concat([
    df_aac, df_paac, df_pssm
], axis=1)
df_combined

## Plots

First, let's create a plot of the three feature types, and see how they look together in a dataframe:

In [ ]:
import re

aac = re.compile(r"^[ACDEFGHIKLMNPQRSTVWY]$")
paac = re.compile(r"^[ACDEFGHIKLMNPQRSTVWY][ACDEFGHIKLMNPQRSTVWY]$")
pssm = re.compile(r"^[ACDEFGHIKLMNPQRSTVWY][ACDEFGHIKLMNPQRSTVWY]_\d\d_\d$")

def get_feature_type(feature_name):
    if paac.match(feature_name):
        return "paac"
    elif aac.match(feature_name):
        return "aac"
    elif pssm.match(feature_name):
        return "pssm"
    else:
        raise ValueError("unknown feature type!")

labeled_clustermap(df_combined.T, df_combined.columns.to_series().apply(get_feature_type))

PAAC and AAC values are a lot lower, since they are percentages, while PSSM values are scaled to [0,1]. The scaling was done along the feature axis, to avoid information sharing. What does the data look like when we also scale PAAC and AAC to [0,1]?

In [ ]:
from sklearn.preprocessing import minmax_scale


def min_max_samples(df_feature):
    return pd.DataFrame(
        data=minmax_scale(df_feature, axis=1),
        index=df_feature.index,
        columns=df_feature.columns,
    )


df_combined_minmax = pd.concat(
    [min_max_samples(df_aac), min_max_samples(df_paac), min_max_samples(df_pssm)],
    axis=1,
)
# df_combined_minmax = pd.DataFrame(data=minmax_scale(df_combined, axis=1), index=df_combined.index, columns = df_combined.columns)
labeled_clustermap(
    df_combined_minmax.T, df_combined_minmax.columns.to_series().apply(get_feature_type)
)
# df_combined_minmax#.iloc[0].max()


There is a cluster with almost exclusively PSSM, and one cluster that is more mixed. The latter contains features with lower values.

The feature PCA plot looks similar to the other organisms and also shows the overlap:

In [ ]:
pca_plot_2d(df_combined_minmax.T, df_combined_minmax.T.index.to_series().apply(get_feature_type).rename("Feature"))

What does the data look like when standardizing along the feature axis? In the model, the parameters for this will be derived from the training set and applied to the test set, to avoid data leakage.

In [ ]:
from sklearn.preprocessing import scale

df_combined_std = pd.DataFrame(
    data=scale(df_combined), index=df_combined.index, columns=df_combined.columns
)
labeled_clustermap(
    df_combined_std.T, df_combined.columns.to_series().apply(get_feature_type)
)
labeled_clustermap(
    df_combined_std, df.keywords_transport
)

The composition data looks a bit noisier, however this could also be caused by the fact that more PSSM features exist, and therefore have a higher impact on the h-clustering algorithm. The second plot shows that the two sample clusters do not correspond to the classes. 

There are two clusters, one for PSSM and one for the compositions. There are sub-clusters taht correspond to the substrates

The PCA plot of the standardized features also looks similar. There is a separation between most PSSM features and the rest of the dataset, possibly because of their lower variance

In [ ]:
pca_plot_2d(df_combined_std.T, df_combined_std.T.index.to_series().apply(get_feature_type).rename("Feature"))

## Independent test set

In [ ]:
X, y, feature_names, sample_names = preprocess_pandas(
    df_combined, labels, return_names=True
)
(
    X_train,
    X_test,
    y_train,
    y_test,
    sample_names_train,
    sample_names_test,
) = get_independent_test_set(X, y, sample_names=sample_names, test_size=0.2)

## Model comparison

In [ ]:
models_quick_compare(X_train, y_train)

## Feature importance

How many top 50 features come from each feature type?

In [ ]:
top50 = get_feature_score(df_combined, labels, remove_zero_variance=True).sort_values(
    "Normalized score", ascending=False
).head(50)
top50.Feature.apply(get_feature_type).value_counts()

Like in A thaliana and yeast, we get a mix again. For human and e coli, the top50 only came from PSSM.

In [ ]:
top50[top50.Feature.apply(get_feature_type) == "pssm"].head(10)

The most important features are again related to the exchange of Methionine for Proline.

In [ ]:
top50[top50.Feature.apply(get_feature_type) == "pssm"].Feature.apply(lambda x: [x[0], x[1]]).explode().value_counts()

The most important amino acid wrt its conservation was His (H) again. This matches the AAC feature importances of A thaliana and yeast. In human and yeast, Pro was the most important.  

## Parameter tuning

#### Custom transformer


Whole dataset, linear svm

In [ ]:
gsearch = optimize_hyperparams(
    X_train,
    y_train,
    kernel="linear",
    dim_reduction=None,
    C=[0.01, 0.1, 1, 10],
)

The PSSMselector selects all pssms, so we get the same score: 

In [ ]:
gsearch = optimize_hyperparams(
    X_train,
    y_train,
    kernel="linear",
    dim_reduction=None,
    C=[0.01, 0.1, 1, 10],
    feature_transformer="pssm",
    feature_names=feature_names,
)

RBF improves the results:

In [ ]:
gsearch = optimize_hyperparams(
    X_train,
    y_train,
    kernel="rbf",
    dim_reduction=None,
    C=[0.1, 1, 10],
)

Removing PSSMs with one iteration leads to a slight improvement:

In [ ]:
gsearch = optimize_hyperparams(
    X_train,
    y_train,
    kernel="rbf",
    dim_reduction=None,
    C=[0.1, 1, 10, 100],
    feature_transformer="pssm",
    feature_names=feature_names,
)
best_estimator_svc = gsearch

## Dimensionality reduction

Trying only in steps of 20, to reduce runtime.

In [ ]:
gsearch = optimize_hyperparams(
    X_train,
    y_train,
    kernel="rbf",
    dim_reduction="kbest",
    C=[1, 10, 100],
    select_k_steps=20,
    remove_zero_var=True,
)

In [ ]:
gsearch = optimize_hyperparams(
    X_train,
    y_train,
    kernel="rbf",
    dim_reduction="pca",
    # feature_transformer="pssm",
    # feature_names=feature_names,
    # C=[0.1, 0.01, 0.001],
    # gamma=[0.1, 0.01, 0.001],
)


Does using the pssm selector change anything?

In [ ]:
gsearch = optimize_hyperparams(
    X_train,
    y_train,
    kernel="rbf",
    dim_reduction="pca",
    feature_transformer="pssm",
    feature_names=feature_names,
    # C=[0.1, 0.01, 0.001],
    # gamma=[0.1, 0.01, 0.001],
)

best_estimator_svc_pca = gsearch

## Validation



In [ ]:
get_confusion_matrix(X_test, y_test, best_estimator_svc_pca, labels=labels)

In [ ]:
get_classification_report(X_test, y_test, best_estimator_svc_pca, labels=labels)

In [ ]:
get_confusion_matrix(X_test, y_test, best_estimator_svc, labels=labels)

In [ ]:
get_classification_report(X_test, y_test, best_estimator_svc, labels=labels)

## Estimating validation variance

How much did the result depend on choosing the training and test sets?

Mean and standard deviation for randomly selected training and validation sets.

In [ ]:
df_scores, df_params = full_test(
    df_combined,
    labels,
    dim_reduction="pca",
    kernel="rbf",
    repetitions=10,
    feature_transformer="pssm",
)
df_scores_gr = df_scores.groupby(["label", "dataset"], as_index=False)
print("Mean F1")
display(df_scores_gr.mean().pivot(index="label", columns="dataset", values="F1 score"))
print("Sdev F1")
display(df_scores_gr.std().pivot(index="label", columns="dataset", values="F1 score"))
print("Parameters")
display(df_params)

The combined feature leads to decent performance, but it does not perform as good as the PSSM feature for the same dataset and the same pipeline.